In [8]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

dev = qml.device("default.qubit", wires=2)

In [9]:
@qml.qnode(dev, interface="autograd")
def C1(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.Identity(0)@qml.X(1))

In [10]:
@qml.qnode(dev, interface="autograd")
def C2(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.X(0)@qml.X(1))

In [11]:
@qml.qnode(dev, interface="autograd")
def C3(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.Y(0)@qml.Y(1))


In [12]:
def cost_fn(param):
    cost = 2 - C1(param) - 0.5*C2(param) - 0.5*C3(param)
    return cost

In [13]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

theta = np.array([0.5*np.pi, 0.5*np.pi, 0.5*np.pi, 0.5*np.pi], requires_grad=True)

# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-04

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f}")
        
    if conv <= conv_tol:
        print(f"Converged at Step = {n},  Energy = {energy[-1]:.8f}")
        break




Step = 0,  Energy = 1.61058166
Step = 2,  Energy = 0.92919395
Step = 4,  Energy = 0.54090589
Step = 6,  Energy = 0.43152534
Step = 8,  Energy = 0.40144038
Step = 10,  Energy = 0.38991480
Step = 12,  Energy = 0.38509493
Step = 14,  Energy = 0.38315260
Step = 16,  Energy = 0.38240480
Step = 18,  Energy = 0.38212581
Converged at Step = 19,  Energy = 0.38206210
